In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
import matplotlib.pyplot as plt
if tf.config.experimental.list_physical_devices('GPU'):
    print('GPU is used')
else:
    print('no GPU available, CPU is used instead')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
np.set_printoptions(precision=4)

In [ ]:
df_train= pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header=None)
df_test=pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv',header=None)
print("Dataframes MITBIH correctly read into workspace")

#split target and value
train_target=df_train[187]
test_target=df_test[187]
train=df_train.drop(187,axis=1)
test=df_test.drop(187,axis=1)

In [ ]:
class Config_Sampling:
    oversample = True #equals to B_SMOTE
    undersample = False
    sample_name = "UNDEFINED_SAMPLE"
    
    

Train_Simple_ANN = False #Trains the simple ANN
Train_Simple_CNN = False #Trains the simple CNN
Train_Advanced_CNN = True #Trains the advanced CNN
Train_Simple_RNN = False #Trains the simple RNN
# TRAIN OTHER MODELS ## Placeholder for switch for other models
 

In [ ]:
oversampler = SMOTE()
undersampler = RandomUnderSampler()

In [ ]:
if Config_Sampling.oversample:
    train, train_target = oversampler.fit_resample(df_train.iloc[:,:-1], df_train.iloc[:,-1])
    Config_Sampling.sample_name = "MITBIH_B_SMOTE"
    print("Sample Name:", Config_Sampling.sample_name)
elif Config_Sampling.undersample:
    train, train_target = undersampler.fit_resample(df_train.iloc[:,:-1], df_train.iloc[:,-1])
    Config_Sampling.sample_name = "MITBIH_C_RUS"
    print("Sample Name:", Config_Sampling.sample_name)
else: 
    print("Using the original mitbih dataset")
    Config_Sampling.sample_name = "MITBIH_A_Original"
    print("Sample Name:", Config_Sampling.sample_name)

In [ ]:
#overview for later comparison (has the model produced the right output?)
print("Initial Train shape", train.shape)
print("Initial Test shape", test.shape)
print("Initial Value count for train dataset (target):", train_target.value_counts())
print("Initial Value count for test dataset (target):", test_target.value_counts())

print("Initial percentages for train dataset (target):", 100* train_target.value_counts() / len(train_target))
print("Initial percentages for test dataset (target):", 100 *test_target.value_counts() / len(test_target))

# **Deep Learning Models**

## **Simple Artificial neural Network**
ANN without convolutional layers. Only Dense layers are used. No Pooling, Flattening or Dropping out. Base model for later comparison.

In [ ]:
if Train_Simple_ANN == True:
    class Config_ANN:
        epochs = 70 #70
        batch_size = 10 #10
        exp_name = 4 #Please WRITE THIS DOWN IN EXCEL FILE (With screenshot)
        patience = 70 #10 # for early stopping
        initial_learning_rate=0.001 #Initial Learning Rate for ADAM: 0.001
        optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
        reduce_lr_every_10_epochs = True #Reduce the learning rate each 10 Epochs with the rate defined below
        lr_reduction_rate = 0.5 # Reduction Rate for reducing each 10 epochs. 0.5 means, that lr is halfed each 10 epochs.
        filepath_checkpoint = '/kaggle/working/Simple_ANN/'+str(Config_Sampling.sample_name) +'/experiment_'+str(exp_name) + '_'+str (Config_Sampling.sample_name) +'.weights.h5'
        filepath_accuracy_plot = '/kaggle/working/Simple_ANN/'+str(Config_Sampling.sample_name) +'/experiment_'+str(exp_name) +  '_'+ str (Config_Sampling.sample_name) +'.accuracy_plot.png'
        filepath_loss_plot = '/kaggle/working/Simple_ANN/'+str(Config_Sampling.sample_name) +'/experiment_'+str(exp_name) +  '_'+str (Config_Sampling.sample_name) +'.loss_plot.png'
        filepath_classification_report = '/kaggle/working/Simple_ANN/'+str(Config_Sampling.sample_name) +'/experiment_'+str(exp_name) +  '_'+str (Config_Sampling.sample_name) +'.classification_report.txt'

    ann_model = tf.keras.models.Sequential()
    ann_model.add(tf.keras.layers.Dense(60, activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape=(187,)))
    ann_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    ann_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    ann_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    ann_model.add(tf.keras.layers.Dense(5, activation='softmax'))

    def lr_scheduler(epoch, lr):
        if Config_ANN.reduce_lr_every_10_epochs and epoch % 10 == 0: # %10 == 0 means each 10 epochs (only then no "rest" after dividing trough 10)
            return lr * Config_ANN.lr_reduction_rate  # reduce the learning rate with the configured rate
        else:
            return lr

    # callback for the learning rate reduction schedule
    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=Config_ANN.filepath_checkpoint,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=Config_ANN.patience)

    ann_model.compile(optimizer=Config_ANN.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    ann_model.summary()

    ann_model_history = ann_model.fit(train, train_target, epochs=Config_ANN.epochs, batch_size = Config_ANN.batch_size, validation_data = (test, test_target), callbacks=[model_checkpoint_callback, early_stop_callback, lr_scheduler_callback])

    plt.plot(ann_model_history.history['accuracy'])
    plt.plot(ann_model_history.history['val_accuracy'])
    plt.legend(["accuracy","val_accuracy"])
    plt.title('Accuracy Vs Val_Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(Config_ANN.filepath_accuracy_plot)
    plt.show()
    plt.close()


    plt.plot(ann_model_history.history['loss'])
    plt.plot(ann_model_history.history['val_loss'])
    plt.legend(["loss","val_loss"])
    plt.title('Loss Vs Val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(Config_ANN.filepath_loss_plot)
    plt.show()
    plt.close()

    predictions = ann_model.predict(test).argmax(axis=1) #directly getting classes instead of probabilities.

    report=classification_report(test_target, predictions, digits=4)
    print(report)
    with open(Config_ANN.filepath_classification_report, 'w') as report_file:
        report_file.write(report)

    print("EVERYTHING FINISHED FOR SIMPLE ANN MODEL!")

else:
    print("Simple ANN Model is not trained and evaluated")
    

## **Simple Convolutional neural Network**
CNN with convolutional layers. The same Dense layers from Simple ANN Model are used. No Pooling, Flattening or Dropping out. Base model for later comparison.

*Notes: We could try different variations of flattening, dropping out, pooling for the CNN-Model. But first, lets see what one or more convolutional layers are doing to our Simple ANN Model. Also it is necessary to understand, what dimensions we give the models to work on and what this implies (in best case: We teach the model to evaluate the heartbeat based on the "graphical" shape, just like a real doctor would do*

*We use a Conv1D layer, because our Input Data is one-dimensional (i.e. is a timeseries). We have sequential and not spatial data.*

** What can we modifiy:
- epochs
- batch_size
- patience
- Conv1_filter_num --> Numbers of watching eyes more or less?
- Conv1_filter_size --> test 1, 3, 5 etc --> Resolution of watching eye more or less.
- Pooling instead of flattening?
_ No. of Neurons in Dense Layers (could be also variated in Simple_ANN Model).
**

**Notes: The .toarray function rounds to 4 digits, this might lead to faster, but less precise results?**


In [ ]:
if Train_Simple_CNN == True:
    class Config_CNN:
        epochs = 70 #70
        batch_size =10 # 10
        exp_name = 4
        patience = 70 #10
        initial_learning_rate=0.001 #Initial Learning Rate for ADAM: 0.001
        optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
        reduce_lr_every_10_epochs = True #Reduce the learning rate each 10 Epochs with the rate defined below
        lr_reduction_rate = 0.5 # Reduction Rate for reducing each 10 epochs. 0.5 means, that lr is halfed each 10 epochs.
        Conv1_filter_num = 32
        Conv1_filter_size = 3
        filepath_checkpoint = '/kaggle/working/Simple_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.weights.h5'
        filepath_accuracy_plot = '/kaggle/working/Simple_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.accuracy_plot.png'
        filepath_loss_plot = '/kaggle/working/Simple_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.loss_plot.png'
        filepath_classification_report = '/kaggle/working/Simple_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.classification_report.txt'

    cnn_model = tf.keras.models.Sequential()
    cnn_model.add(tf.keras.layers.Conv1D(Config_CNN.Conv1_filter_num, Config_CNN.Conv1_filter_size, activation='relu', input_shape=(187, 1))) # We add one Conv1D layer to the model
    cnn_model.add(tf.keras.layers.Flatten()) # After 
    cnn_model.add(tf.keras.layers.Dense(60, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    cnn_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    cnn_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    cnn_model.add(tf.keras.layers.Dense(5, activation='softmax'))
    
    def lr_scheduler(epoch, lr):
        if Config_ANN.reduce_lr_every_10_epochs and epoch % 10 == 0: # %10 == 0 means each 10 epochs (only then no "rest" after dividing trough 10)
            return lr * Config_ANN.lr_reduction_rate  # reduce the learning rate with the configured rate
        else:
            return lr

    # callback for the learning rate reduction schedule
    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=Config_CNN.filepath_checkpoint,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)

    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=Config_CNN.patience)

    cnn_model.compile(optimizer=Config_CNN.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    cnn_model.summary()

    # Dfs need to be np-array
    #now we transform Dataframes into numpy arrays
    train_array = train.to_numpy()
    test_array = test.to_numpy()
    print("Train Dataframe shape (before reshaping):", train.shape)
    print("Train Dataframe first 10 rows for comparison:")
    #display(train.head(10)) # display(train.head(10)[0]) for first element in first row
    #display(train.head(10))
    print("train array shape before reshaping:", train_array.shape)
    print("Train Array before reshaping first 10 rows for comparison:")
    #display(train_array[:10]) #display(train_array[:10][0][0]) for first element in first row
    #display(train_array[:10])

    # Reshape of np arrays for conv1D layer in cnn model
    train_reshaped = train_array.reshape(train_array.shape[0], train_array.shape[1], 1)
    test_reshaped = test_array.reshape(test_array.shape[0], test_array.shape[1], 1)
    print("train array shape after reshaping:", train_reshaped.shape)
    print("Train Array after reshaping first 10 rows for comparison:")
    #display(train_reshaped[:10]) #display(train_reshaped[:10][0][0]) for first element of first row
    
    # Rest of code is more or less copied from simple_ANN
    cnn_model_history = cnn_model.fit(train_reshaped, train_target, epochs=Config_CNN.epochs, batch_size=Config_CNN.batch_size, validation_data=(test_reshaped, test_target), callbacks=[model_checkpoint_callback, early_stop_callback, lr_scheduler_callback])
    #cnn_model_history = cnn_model.fit(train.reshape(train.shape[0], train.shape[1], 1), train_target, epochs=Config_CNN.epochs, batch_size=Config_CNN.batch_size, validation_data=(test.reshape(test.shape[0], test.shape[1], 1), test_target), callbacks=[model_checkpoint_callback, early_stop_callback])

    plt.plot(cnn_model_history.history['accuracy'])
    plt.plot(cnn_model_history.history['val_accuracy'])
    plt.legend(["accuracy","val_accuracy"])
    plt.title(f'Accuracy Vs Val_Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(Config_CNN.filepath_accuracy_plot)
    plt.show()
    plt.close()

    plt.plot(cnn_model_history.history['loss'])
    plt.plot(cnn_model_history.history['val_loss'])
    plt.legend(["loss","val_loss"])
    plt.title('Loss Vs Val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(Config_CNN.filepath_loss_plot)
    plt.show()
    plt.close()

    predictions = cnn_model.predict(test_reshaped).argmax(axis=1)

    report = classification_report(test_target, predictions, digits=4)
    print(report)
    with open(Config_CNN.filepath_classification_report, 'w') as report_file:
        report_file.write(report)

    print("EVERYTHING FINISHED FOR SIMPLE CNN MODEL!")

else:
    print("Simple CNN Model is not trained and evaluated")


## **Advanced CNN**

This is the Model from Hakans notebook, it will be adapted, so that the modelstructure can be used with exp1 to exp4.
Hakan can do some notes here ################

Most important change(s):
- Sparse_Categorical_Crossentropy instead of categorical --> Should not change metrics, but removes the need for onehotencoding the dataset, which is not done in this notebook. We have numbers of 0 to 4 for each category, so its wondering, why normal categorical_cross_entropy even worked in the original notebook from hakan?
- By using Sparse_Categorical_Crossentropy i had to adjust the making of the classification report: test_target is now used directly and not with .argmax(axis=1). I also don´t quite understand, why this should be used in any way, because test_target is already a set of predicted classes and doesnt need to be further processed? 
-  Removed the plateu learning rate callback, we can use this in experiment5, but it was not used during exp1 to 4

In [ ]:
if Train_Advanced_CNN == True:
    class Config_Advanced_CNN:
        epochs = 70 #70
        batch_size =10 # 10
        exp_name = 4
        patience = 70 #10
        initial_learning_rate=0.001 #Initial Learning Rate for ADAM: 0.001
        optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
        reduce_lr_every_10_epochs = True #Reduce the learning rate each 10 Epochs with the rate defined below
        lr_reduction_rate = 0.5 # Reduction Rate for reducing each 10 epochs. 0.5 means, that lr is halfed each 10 epochs.
        Conv1_filter_num = 32
        Conv1_filter_size = 3
        filepath_checkpoint = '/kaggle/working/Advanced_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.weights.h5'
        filepath_accuracy_plot = '/kaggle/working/Advanced_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.accuracy_plot.png'
        filepath_loss_plot = '/kaggle/working/Advanced_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.loss_plot.png'
        filepath_classification_report = '/kaggle/working/Advanced_CNN/'+str(Config_Sampling.sample_name)+'/experiment_'+str(exp_name)+'_'+str(Config_Sampling.sample_name)+'.classification_report.txt'

    adv_cnn_model = tf.keras.models.Sequential()
    adv_cnn_model.add(tf.keras.layers.Conv1D(Config_Advanced_CNN.Conv1_filter_num, Config_Advanced_CNN.Conv1_filter_size, activation='relu', input_shape=(187, 1))) # We add one Conv1D layer to the model
    adv_cnn_model.add(tf.keras.layers.MaxPooling1D(pool_size=3, strides=2)) # We add one Conv1D layer to the model
    adv_cnn_model.add(tf.keras.layers.Conv1D(Config_Advanced_CNN.Conv1_filter_num//2, Config_Advanced_CNN.Conv1_filter_size, activation='relu' )) # We add one Conv1D layer to the model
    adv_cnn_model.add(tf.keras.layers.MaxPooling1D(pool_size=3, strides=2)) # We add one Conv1D layer to the model
    adv_cnn_model.add(tf.keras.layers.Flatten()) # After  
    adv_cnn_model.add(tf.keras.layers.Dropout(rate=0.2))
    adv_cnn_model.add(tf.keras.layers.Dense(120, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    adv_cnn_model.add(tf.keras.layers.Dense(60, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    adv_cnn_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    adv_cnn_model.add(tf.keras.layers.Dense(5, activation='softmax'))
    
    def lr_scheduler(epoch, lr):
        if Config_Advanced_CNN.reduce_lr_every_10_epochs and epoch % 10 == 0: # %10 == 0 means each 10 epochs (only then no "rest" after dividing trough 10)
            return lr * Config_Advanced_CNN.lr_reduction_rate  # reduce the learning rate with the configured rate
        else:
            return lr

    # callback for the learning rate reduction schedule
    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
    
    # Callback for the model checkpoint
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=Config_Advanced_CNN.filepath_checkpoint,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    #Callback for the LR Plateu --> Not used in exp1 to exp4 and therefore commented out, but could be a really good setup for exp5 with all models?
    """LR_plateu_callback = ReduceLROnPlateau(monitor = 'val_acc', min_delta = 0.001, patience = 20,
                                           factor = 0.5, cooldown = 0, verbose = 1)"""
    
    # Normal early stop callback
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=Config_Advanced_CNN.patience)

    adv_cnn_model.compile(optimizer=Config_Advanced_CNN.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    adv_cnn_model.summary()

    # Dfs need to be np-array
    #now we transform Dataframes into numpy arrays
    train_array = train.to_numpy()
    test_array = test.to_numpy()
    print("Train Dataframe shape (before reshaping):", train.shape)
    print("Train Dataframe first 10 rows for comparison:")
    #display(train.head(10)) # display(train.head(10)[0]) for first element in first row
    #display(train.head(10))
    print("train array shape before reshaping:", train_array.shape)
    print("Train Array before reshaping first 10 rows for comparison:")
    #display(train_array[:10]) #display(train_array[:10][0][0]) for first element in first row
    #display(train_array[:10])

    # Reshape of np arrays for conv1D layer in cnn model
    train_reshaped = train_array.reshape(train_array.shape[0], train_array.shape[1], 1)
    test_reshaped = test_array.reshape(test_array.shape[0], test_array.shape[1], 1)
    print("train array shape after reshaping:", train_reshaped.shape)
    print("Train Array after reshaping first 10 rows for comparison:")
    #display(train_reshaped[:10]) #display(train_reshaped[:10][0][0]) for first element of first row
    
    # We do not use LR_plateu_callback in exp1 to exp4 and therefore remove it for now (exp5 maybe?)
    adv_cnn_model_history = adv_cnn_model.fit(train_reshaped, train_target, epochs=Config_Advanced_CNN.epochs, batch_size=Config_Advanced_CNN.batch_size, 
                                      validation_data=(test_reshaped, test_target), 
                                      callbacks=[model_checkpoint_callback, early_stop_callback, lr_scheduler_callback]) #, LR_plateu_callback
    
    plt.plot(adv_cnn_model_history.history['accuracy'])
    plt.plot(adv_cnn_model_history.history['val_accuracy'])
    plt.legend(["accuracy","val_accuracy"])
    plt.title(f'Accuracy Vs Val_Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(Config_Advanced_CNN.filepath_accuracy_plot)
    plt.show()
    plt.close()

    plt.plot(adv_cnn_model_history.history['loss'])
    plt.plot(adv_cnn_model_history.history['val_loss'])
    plt.legend(["loss","val_loss"])
    plt.title('Loss Vs Val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(Config_Advanced_CNN.filepath_loss_plot)
    plt.show()
    plt.close()

    predictions = adv_cnn_model.predict(test_reshaped).argmax(axis=1)
    
    ## DEBUGGING ERROR ON MAKING CLASSIFICATION REPORT
    print("predictions from adv_cnn_model:")
    display(predictions)
    print("")
    print("test target from adv_cnn_model:") ###WHY IS TEST TARGET ALSO USED WITH argmax function? is this the same in the other codes? TGest target is NOT used with argtmax in other code.
    display(test_target)
    print("")
    print(f"Shape of predictions: {predictions.shape}, shape of test target: {test_target.shape} in advanced CNN Model")
    
    report = classification_report(test_target, predictions, digits=4) # test_target used directly without .argmax(axis=1) @Hakan, why did you do this in your notebook?
    print(report)
    with open(Config_Advanced_CNN.filepath_classification_report, 'w') as report_file:
        report_file.write(report)

    print("EVERYTHING FINISHED FOR ADVANCED CNN MODEL!")

else:
    print("ADVANCED CNN Model is not trained and evaluated")

## **Simple RNN**
Just the ANN and the first Dense layer is an simple_RNN layer with the same amount of (hidden-)units. The RNN layer should work as follows (doubleckeck before report): The (simple_)RNN layer can be viewed as similar to a dense layer but with the ability to store and update its own internal state based on previous inputs. This allows it to retain information from past time steps and incorporate it into the calculation of the current output. While a dense layer performs a linear transformation on each input independently, the RNN layer's internal state is updated recursively, allowing it to model sequential dependencies.
#### On the changed input shape of the simple_RNN layer
The input shape was changed to (None, 187) to accommodate sequences of variable length, allowing the RNN model to process EKG signals with different sequence lengths efficiently. While the data type remains consistent (EKG signals), this change enables flexibility in handling sequences of varying lengths, which is common in time series data like EKG signals. I.e.: the RNN layer expects this kind of definition for the input shape despite the fact, that we fed it data of the same dimensions all the time.
Just as for simple_CNN, we have to adjust our input shape to accomodate for the requirements of the specific first layer.

In [ ]:
if Train_Simple_RNN == True:  
    class Config_RNN:
        epochs = 70
        batch_size = 10
        exp_name = 1
        patience = 10  # for early stopping
        initial_learning_rate = 0.001 #0.001 is default for ADAM
        optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
        reduce_lr_every_10_epochs = False
        lr_reduction_rate = 0.5
        filepath_checkpoint = '/kaggle/working/Simple_RNN/' + str(Config_Sampling.sample_name) + '/experiment_' + str(
            exp_name) + '_' + str(Config_Sampling.sample_name) + '.weights.h5'
        filepath_accuracy_plot = '/kaggle/working/Simple_RNN/' + str(Config_Sampling.sample_name) + '/experiment_' + str(
            exp_name) + '_' + str(Config_Sampling.sample_name) + '.accuracy_plot.png'
        filepath_loss_plot = '/kaggle/working/Simple_RNN/' + str(Config_Sampling.sample_name) + '/experiment_' + str(
            exp_name) + '_' + str(Config_Sampling.sample_name) + '.loss_plot.png'
        filepath_classification_report = '/kaggle/working/Simple_RNN/' + str(Config_Sampling.sample_name) + '/experiment_' + str(
            exp_name) + '_' + str(Config_Sampling.sample_name) + '.classification_report.txt'
    
    # Dfs need to be np-array
    #now we transform Dataframes into numpy arrays
    train_array = train.to_numpy()
    test_array = test.to_numpy()
    print("Train Dataframe shape (before reshaping):", train.shape)
    print("Train Dataframe first 10 rows for comparison:")
    #display(train.head(10)) # display(train.head(10)[0]) for first element in first row
    #display(train.head(10))
    print("train array shape before reshaping:", train_array.shape)
    print("Train Array before reshaping first 10 rows for comparison:")
    #display(train_array[:10]) #display(train_array[:10][0][0]) for first element in first row
    #display(train_array[:10])

    # Reshape of np arrays for conv1D layer in RNN model
    train_reshaped = train_array.reshape(train_array.shape[0], train_array.shape[1], 1)
    test_reshaped = test_array.reshape(test_array.shape[0], test_array.shape[1], 1)
    print("train array shape after reshaping:", train_reshaped.shape)
    print("Train Array after reshaping first 10 rows for comparison:")
    #display(train_reshaped[:10]) #display(train_reshaped[:10][0][0]) for first element of first row
    
    rnn_model = tf.keras.models.Sequential()
    rnn_model.add(tf.keras.layers.SimpleRNN(60, activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape=(train_reshaped.shape[1], train_reshaped.shape[2])))  # Adjusted input shape for RNN
    rnn_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    rnn_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    rnn_model.add(tf.keras.layers.Dense(20, activation=tf.keras.layers.LeakyReLU(alpha=0.001)))
    rnn_model.add(tf.keras.layers.Dense(5, activation='softmax'))

    def lr_scheduler(epoch, lr):
        if Config_RNN.reduce_lr_every_10_epochs and epoch % 10 == 0:
            return lr * Config_RNN.lr_reduction_rate
        else:
            return lr

    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=Config_RNN.filepath_checkpoint,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=Config_RNN.patience)

    rnn_model.compile(optimizer=Config_RNN.optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    rnn_model.summary()
    
    rnn_model_history = rnn_model.fit(train_reshaped, train_target, epochs=Config_RNN.epochs, batch_size=Config_RNN.batch_size,
                                      validation_data=(test_reshaped, test_target),
                                      callbacks=[model_checkpoint_callback, early_stop_callback, lr_scheduler_callback])

    plt.plot(rnn_model_history.history['accuracy'])
    plt.plot(rnn_model_history.history['val_accuracy'])
    plt.legend(["accuracy", "val_accuracy"])
    plt.title('Accuracy Vs Val_Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.savefig(Config_RNN.filepath_accuracy_plot)
    plt.show()
    plt.close()


    plt.plot(rnn_model_history.history['loss'])
    plt.plot(rnn_model_history.history['val_loss'])
    plt.legend(["loss", "val_loss"])
    plt.title('Loss Vs Val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig(Config_RNN.filepath_loss_plot)
    plt.show()
    plt.close()

    predictions = rnn_model.predict(test).argmax(axis=1)

    report = classification_report(test_target, predictions, digits=4)
    print(report)
    with open(Config_RNN.filepath_classification_report, 'w') as report_file:
        report_file.write(report)

    print("EVERYTHING FINISHED FOR SIMPLE RNN MODEL!")

else:
    print("Simple RNN Model is not trained and evaluated")
